In [10]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import preprocfs as pre
from hyperopt import Trials,STATUS_OK,tpe
from hyperas import optim
from hyperas.distributions import choice,uniform

In [11]:
def data():
    tf_seed = 720
    np_seed = 1995
    dev_proportion = 10/100
    data_path = 'ConstrictionDegreeOnly/task2maeda_a_zero/data/'
    # import data into dataframe and verify dataframe dimensions
    x_train_dev = pd.read_csv(data_path + 'x_train.csv',index_col=0,header=0)
    x_cols = x_train_dev.columns
    y_train_dev = pd.read_csv(data_path + 'y_train.csv',index_col=0,header=0)
    y_cols = y_train_dev.columns
    x_train_raw = pd.read_csv(data_path + 'x_train_raw.csv',index_col=0,header=0).to_numpy()
    y_train_raw = pd.read_csv(data_path + 'y_train_raw.csv',index_col=0,header=0).to_numpy()
    # repeatability
    tf.random.set_seed(tf_seed)
    np.random.seed(np_seed)

    # split data
    [x_train,y_train,x_dev,y_dev] = pre.split_data(x_train_dev,y_train_dev,dev_proportion,numpy=True)
    return x_train,y_train,x_dev,y_dev

In [12]:
# Hyperparameters and model architecture
def create_model():
    epochs = 600
    batch_size = {{choice([64,128,256])}}
    n_hidden_layers = {{choice([3,4,5])}}
    n_units_per_layer = {{choice([128,256,512])}}
    act = 'relu'
    learning_rate = {{choice([0.01,0.001,0.0001])}}
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    loss_function = 'mse'
    dev_proportion = 10/100
    tf_seed = 720
    np_seed = 1995
    # repeatability
    tf.random.set_seed(tf_seed)
    np.random.seed(np_seed)
    # build model
    model = tf.keras.Sequential()
    model.add(tf.keras.Input(x_train.shape[1],))
    for n in range(n_hidden_layers):
        model.add(tf.keras.layers.Dense(n_units_per_layer,activation=act))
    model.add(tf.keras.layers.Dense(y_train.shape[1]))
    model.compile(optimizer=optimizer, loss=loss_function)
    history = model.fit(x_train,y_train,validation_data=(x_dev,y_dev),batch_size=batch_size,epochs=epochs,verbose=False)
    loss = model.evaluate(x_dev,y_dev,batch_size=batch_size,verbose=0)
    return {'loss': loss, 'status': STATUS_OK, 'model': model}

In [13]:
x_train,y_train,x_val,y_val = data()
best_run, best_model = optim.minimize(
                        model=create_model,
                        data=data,
                        algo=tpe.suggest,
                        max_evals=50,
                        trials=Trials(),
                        notebook_name='hyperparameter-optimization'
)
print(best_run)

>>> Imports:
#coding=utf-8

try:
    import tensorflow as tf
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    import matplotlib.pyplot as plt
except:
    pass

try:
    import time
except:
    pass

try:
    import preprocfs as pre
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform
except:
    pass

>>> Hyperas search space:

def get_space():
    return {
        'batch_size': hp.choice('batch_size', [64,128,256]),
        'n_hidden_layers': hp.choice('n_hidden_layers', [3,4,5]),
        'n_units_per_layer': hp.choice('n_units_per_layer', [128,256,512]),
        'learning_rate': hp.choice('learning_rate', [0.01,0.001,0.0001]),
    }

>>> Data
  1: 
  2: tf_seed = 720
  3: np_seed = 1995
  4: dev_proportion = 10/100
  5: data_path = 'ConstrictionDegreeOnly/task2maeda_a_

In [14]:
data_path = 'ConstrictionDegreeOnly/task2maeda_a_zero/data/'
x_test = pd.read_csv(data_path + 'x_test.csv',index_col=0,header=0).to_numpy()
y_test = pd.read_csv(data_path + 'y_test.csv',index_col=0,header=0).to_numpy()
print(best_model.evaluate(x_test,y_test))

292860/292860 [==============================] - 11s 37us/sample - loss: 3.2640e-05
3.263997600156943e-05
